# 16.1 Char-RNN을 사용해 셰익스피어 문체 생성하기

## 16.1.1 훈련 데이터셋 만들기

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# 셰익스피어 작품 모두 다운로드
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1130496/1115394 [==============================] - 0s 0us/step


In [ ]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [ ]:
# 모든 글자를 정수로 인코딩
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) # char_level : 글자 수준 인코딩
tokenizer.fit_on_texts(shakespeare_text)

In [ ]:
tokenizer.texts_to_sequences(["First"]) # 숫자로

[[20, 6, 9, 8, 3]]

In [ ]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]]) # 문자로

['f i r s t']

In [ ]:
max_id = len(tokenizer.word_index) # 고유 글자 개수
dataset_size = tokenizer.document_count # 전체 글자 개수

In [ ]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1 # 전체 텍스트를 인코딩하여 각 글자를 ID로 나타내기, 1에서 39까지 대신 0에서 38까지 ID를 얻기 위해 -1

In [ ]:
[encoded]

[array([19,  5,  8, ..., 20, 26, 10])]

## 16.1.2 순차 데이터셋을 나누는 방법

#### 시계열을 훈련 셋, 검증 셋, 테스트 셋으로 나누는 것은 간단한 작업 X
#### 주어진 문제에 따라 달라짐

In [ ]:
# 여기에서는 텍스트의 처음 90%를 훈련 세트로 사용
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

## 16.1.3 순차 데이터를 윈도 여러 개로 자르기

In [ ]:
n_steps = 100
window_length = n_steps + 1 # target = 1글자 앞의 input
dataset = dataset.window(window_length, shift=1, drop_remainder=True) # shift=1 : 가장 큰 훈련 세트를 만들 수 있음, drop_remainer=True : 모든 윈도가 동일한 글자수를 포함하도록

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length)) # 훈련에 중첩 데이터셋을 바로 사용할 수 없음 -> 플랫 데이테 셋으로 만들기 ex) {{1, 2}, {3, 4, 5, 6}} = {1, 2, 3, 4, 5, 6}

In [ ]:
# 윈도를 배치로 만들기 및 입력과 타깃을 분리하기
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [ ]:
# 범주형 입력 특성 -> 원핫 벡터나 임베딩으로 인코딩 해야 함
# 여기서는 고유한 그자 수가 적기 때문에 원핫 벡터 이용
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [ ]:
# 프리페칭 추가
dataset = dataset.prefetch(1)

## 16.1.4 Char-RNN 모델 만들고 훈련하기

#### 이전 글자 100개를 기반으로 다음 글자를 예측
- 유닛 128개를 가진 GRU 층 2개
- 입력과 은닉 상태에 20% 드롭아웃 이용

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     #dropout=0.2, recurrent_dropout=0.2), # 순환 드롭아웃 : 너무 느려요ㅠㅠㅠ
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

Epoch 1/10
31368/31368 [==============================] - 1627s 52ms/step - loss: 1.6210
Epoch 2/10
31368/31368 [==============================] - 1620s 52ms/step - loss: 1.5351
Epoch 3/10
31368/31368 [==============================] - 1621s 52ms/step - loss: 1.5124
Epoch 4/10
31368/31368 [==============================] - 1623s 52ms/step - loss: 1.5002
Epoch 5/10
31368/31368 [==============================] - 1623s 52ms/step - loss: 1.4923
Epoch 6/10
31368/31368 [==============================] - 1636s 52ms/step - loss: 1.4871
Epoch 7/10
31368/31368 [==============================] - 1648s 52ms/step - loss: 1.4828
Epoch 8/10
31368/31368 [==============================] - 1644s 52ms/step - loss: 1.4798
Epoch 9/10
31368/31368 [==============================] - 1642s 52ms/step - loss: 1.4767
Epoch 10/10
31368/31368 [==============================] - 1647s 52ms/step - loss: 1.4750


## 16.1.5 Char-RNN 모델 사용하기

In [ ]:
# 새로운 텍스트 넣으려면?! 전처리 함수
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [ ]:
X_new = preprocess(["How are yo"])
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 첫 번째 문장, 마지막 글자

'u'

## 16.1.6 가짜 셰익스피어 텍스트 생성하기

#### 새로운 텍스트 생성?
1. 초기 텍스트 입력
2. 모델이 가장 가능성 있는 다음 글자 예측
3. 예측된 글자를 가지고 늘어난 텍스트를 모델에 전달하여 다음 글자 예측
But, 같은 단어가 반복되는 경우가 많음  
=> tf.random.categorical() 함수를 사용해 모델이 추정한 확률을 기반으로 다음 글자를 무작위로 선택 O

In [ ]:
# 다음 글자를 선택하고 입력 텍스트에 추가
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature # 생성된 텍스트의 다양성을 더 많이 제어하려면? 온도라고 불리는 숫자를 로짓으로 나눔 (0에 가까울수록 높은 확률을 가진 글자를 선택)
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
print(complete_text("t", temperature=0.2))

the man of all the rest was the words of signior ba


In [ ]:
print(complete_text("t", temperature=1))

t od sirrah, so please thee wiolds:
i wrong'd to be


In [ ]:
print(complete_text("t", temperature=2))

tem face? tigli, higlle's gotd hibhrel.-
ane himn a


#### 조금 더 좋은 성능?
- GRU 층과 층의 뉴런 수를 늘리고 더 오래 훈련하거나 규제(recurrent_dropout=0.3) 추가

- 현재 모델은 100보다 긴 패턴 학습 X
- 윈도를 크게 할 수 있지만 학습 어려워짐
- LSTM과 GRU 셀이라도 매우 긴 시퀀스는 다룰 수 X
- 아니면 상태가 있는 RNN 사용

## 16.1.7 상태가 있는 RNN

- RNN이 한 훈련 배치를 처리한 후에 마지막 상태를 다음 훈련 배치의 초기 상태로 사용?   -> 역전파는 짧은 시퀀스에서 일어나지만 모델이 장기간 패턴을 학습할 수 있음  
=> 상태가 있는 RNN

- 상태가 있는 RNN 만드는 방법
1. 순차적이고 겹치지 않는 입력 시퀀스 만들기  
Why? 배치에 있는 각 입력 시퀀스가 이전 배치의 시퀀스가 끝난 지점에서 시작해야 함
2. Dataset을 만들 때 window() 메서드에서 shift=n_steps 사용
3. shuffle() 메서드 호출 X

In [ ]:
# 하나의 윈도를 갖는 배치를 만들어야 함
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
# 상태가 있는 RNN 모델 생성시 포인트!
# stateful=True 지정해야 함
# 배치 크기 알아야 함 => 첫 번째 층에 batch_input_shape 매개변수를 지정해야 함
# 입력은 어떤 길이도 가질 수 있으므로 두 번째 차원은 지정하지 않아도 됨
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [ ]:
# 에포크 끝마다 텍스트를 다시 시작하기 전에 상태를 재설정 해야 함
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
      6/Unknown - 3s 54ms/step - loss: 3.6243WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0220s vs `on_train_batch_end` time: 0.0264s). Check your callbacks.
313/313 [==============================] - 19s 52ms/step - loss: 2.6270
Epoch 2/50
313/313 [==============================] - 16s 52ms/step - loss: 2.2420
Epoch 3/50
313/313 [==============================] - 16s 52ms/step - loss: 2.1096
Epoch 4/50
313/313 [==============================] - 16s 52ms/step - loss: 2.0326
Epoch 5/50
313/313 [==============================] - 16s 52ms/step - loss: 1.9810
Epoch 6/50
313/313 [==============================] - 16s 53ms/step - loss: 1.9434
Epoch 7/50
313/313 [==============================] - 16s 52ms/step - loss: 1.9154
Epoch 8/50
313/313 [==============================] - 16s 52ms/step - loss: 1.8943
Epoch 9/50
313/313 [==============================] - 16s 52ms/step - loss: 1.8766
Epoch 10/50
313/313 [=============

#### 이 모델은 훈련할 때 사용한 것과 동일한 크기의 배치로만 예측이 가능! 이러한 제약을 없애기 위해서는 동일한 구조의 상태가 없는 모델을 만들고 상태가 있는 모델의 가중치를 복사하면 됨